In [ ]:
!pip3 install catboost

     |████████████████████████████████| 76.1 MB 1.3 MB/s 


In [1]:
import pandas as pd
import numpy as np

csv_path = "trades-df-2021_11_1_0_0_0-2021_12_21_0_0_0.csv"
df = pd.read_csv(csv_path)

In [11]:
x_columns = list(filter(lambda column: not 'punch' in column and not 'weighted-moving-average' in column and not 'exp-moving-average' in column, df.columns))
y_columns = list(filter(lambda column: 'punch' in column, df.columns))
print(x_columns)
print(y_columns)

['seconds-since-midnight', 'seconds-since-1-trades-ago-BUY', 'seconds-since-10-trades-ago-BUY', 'seconds-since-50-trades-ago-BUY', 'seconds-since-100-trades-ago-BUY', 'seconds-since-1000-trades-ago-BUY', 'trade-amount-BUY-600-sec', 'trade-volume-BUY-600-sec', 'open-close-diff-BUY-600-sec', 'moving-average-BUY-600-sec', 'stochastic-oscillator-BUY-600-sec', 'trade-amount-BUY-60-sec', 'trade-volume-BUY-60-sec', 'open-close-diff-BUY-60-sec', 'moving-average-BUY-60-sec', 'stochastic-oscillator-BUY-60-sec', 'trade-amount-BUY-30-sec', 'trade-volume-BUY-30-sec', 'open-close-diff-BUY-30-sec', 'moving-average-BUY-30-sec', 'stochastic-oscillator-BUY-30-sec', 'trade-amount-BUY-10-sec', 'trade-volume-BUY-10-sec', 'open-close-diff-BUY-10-sec', 'moving-average-BUY-10-sec', 'stochastic-oscillator-BUY-10-sec', 'trade-amount-BUY-1-sec', 'trade-volume-BUY-1-sec', 'open-close-diff-BUY-1-sec', 'moving-average-BUY-1-sec', 'stochastic-oscillator-BUY-1-sec', 'seconds-since-1-trades-ago-SELL', 'seconds-since-1

In [3]:
X = df[x_columns]
secs = list(reversed(sorted(list(set(map(lambda x : int(''.join(x)), [[s for s in column if s.isdigit()] for column in x_columns]))))))

for side in ['BUY', 'SELL']:
  for s in secs[:-1]:
    X[f'trades-{side}-{str(s)}-sec'] -= X[f'trades-{side}-{str(s - (secs[0] - secs[1]))}-sec']
X

ValueError: invalid literal for int() with base 10: ''

In [12]:
commision = 0.0002
punch = commision * 4

y = pd.DataFrame(
    df[y_columns].apply(
        # lambda row : 1 if max(list(row.to_numpy()), key=abs) > punch else -1 if max(list(row.to_numpy()), key=abs) < -punch else 0,
        lambda row : 1 if max(abs(row.to_numpy())) > punch else 0,
        axis = 1
    )
)
y = y.rename({0:'target'}, axis='columns')
print(y)
print(y.value_counts())

      target
0          0
1          0
2          1
3          0
4          0
...      ...
6559       1
6560       0
6561       0
6562       0
6563       1

[6564 rows x 1 columns]
target
0         4304
1         2260
dtype: int64


In [13]:
from catboost import CatBoostClassifier, Pool, metrics, cv
from sklearn.metrics import accuracy_score

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, shuffle = True, random_state=12)

X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, train_size=0.75, shuffle = True, random_state=6)

In [15]:
params = {
    'iterations': 500,
    'learning_rate': 0.1,
    # 'depth': 5,
    'custom_loss': [metrics.Accuracy()],
    'eval_metric': metrics.Accuracy(),
    'random_seed': 42,
    'logging_level': 'Silent',
    'loss_function': 'Logloss',
}
train_pool = Pool(X_train, y_train)
validate_pool = Pool(X_validation, y_validation)

model = CatBoostClassifier(**params)
model.fit(
    train_pool,
    eval_set=validate_pool,
    plot=True,
)
print('Model validation accuracy: {:.4}'.format(
    accuracy_score(y_test, model.predict(X_test))
))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Model validation accuracy: 0.773


In [16]:
cv_params = model.get_params()
cv_params.update({
    'loss_function': metrics.MultiClass()
})
cv_data = cv(
    Pool(X, y),
    cv_params,
    plot=True
)
print('Best validation accuracy score: {:.2f}±{:.2f} on step {}'.format(
    np.max(cv_data['test-Accuracy-mean']),
    cv_data['test-Accuracy-std'][np.argmax(cv_data['test-Accuracy-mean'])],
    np.argmax(cv_data['test-Accuracy-mean'])
))
print('Precise validation accuracy score: {}'.format(np.max(cv_data['test-Accuracy-mean'])))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Best validation accuracy score: 0.78±0.00 on step 494
Precise validation accuracy score: 0.776356963550407


In [17]:
predictions = model.predict(X_test)
predictions = predictions.reshape(predictions.shape[0], 1)
predictions_probs = model.predict_proba(X_test)
print(predictions[10:20])
print(predictions_probs[10:20])
print(y_test[10:20])

[[0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]]
[[0.74838222 0.25161778]
 [0.70112827 0.29887173]
 [0.74800797 0.25199203]
 [0.91221826 0.08778174]
 [0.96961565 0.03038435]
 [0.00181163 0.99818837]
 [0.9287105  0.0712895 ]
 [0.79310302 0.20689698]
 [0.05961957 0.94038043]
 [0.96201868 0.03798132]]
      target
3657       0
5390       1
6548       1
5761       0
2689       0
4132       1
4224       1
6553       1
4521       1
2016       0


In [18]:
feature_importances = model.get_feature_importance(train_pool)
feature_names = X_train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))


seconds-since-50-trades-ago-BUY: 5.235436699849289
seconds-since-50-trades-ago-SELL: 4.512211277308107
seconds-since-midnight: 3.2807330884591956
open-close-diff-SELL-60-sec: 3.0444555709199044
seconds-since-10-trades-ago-SELL: 3.021588497463507
trade-volume-BUY-30-sec: 2.9591959128743706
open-close-diff-BUY-600-sec: 2.7787033593067667
trade-amount-BUY-30-sec: 2.573732473283224
trade-volume-SELL-60-sec: 2.4586573815016677
trade-volume-SELL-10-sec: 2.4371131672598927
trade-amount-SELL-600-sec: 2.3939949358588675
trade-volume-SELL-600-sec: 2.3844039600123956
trade-amount-SELL-60-sec: 2.3386407276802967
trade-amount-BUY-600-sec: 2.3115305701157696
trade-volume-BUY-600-sec: 2.2778635755696786
seconds-since-100-trades-ago-SELL: 2.1260923800944473
open-close-diff-BUY-60-sec: 2.1237543267064587
seconds-since-1-trades-ago-SELL: 2.1138743791816457
seconds-since-10-trades-ago-BUY: 2.0982953764865866
open-close-diff-SELL-10-sec: 2.047211400531122
trade-amount-BUY-60-sec: 2.0454249239513347
trade-